In [1]:

import numpy as np 
import pandas as pd 
import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
import os
import pickle

DATA_PATH = "Vegetable Images/"
IMG_SIZE = 64
BUILT_DB=False


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")
    
    

Running on the GPU


In [3]:
def preprocess_image(path, augment=False):
    images=[]
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    if augment:
        img2 = cv2.flip(img, 1)
        images = np.array([img, img2])/255.0
    else:
        images = np.array(img)/255.0
    #plt.imshow(img)
    #plt.show()
    return images
    
    

In [4]:
if not BUILT_DB:
    training_data = []
    training_labels = []
    classes={}
    class_id=-1
    for dirname, _, filenames in os.walk(DATA_PATH + "train/"):
        class_name = dirname.split("/")[-1]
        if not class_name in classes and len(class_name)>0:
            class_id+=1
            classes[class_name]=class_id
            print(class_name, class_id)
        for filename in filenames:
            resulting_images = preprocess_image(os.path.join(dirname, filename), augment=True)
            for pic in resulting_images:
            
                training_data.append(pic)
                training_labels.append(class_id)
            
    
    np_training_data = np.array(training_data)
    np_training_labels = np.array(training_labels)

   
    np.save("training_data.npy", np_training_data)
    np.save("training_labels.npy", np_training_labels)
    with open('class_dictionary.pkl', 'wb') as f:
        pickle.dump(classes, f)
        
    BUILT_DB=True
else:
    
    np_training_data = np.load("training_data.npy", allow_pickle=True)
    np_training_labels = np.load("training_labels.npy", allow_pickle=True)
    with open('class_dictionary.pkl', 'rb') as f:
        classes = pickle.load(f)

Bean 0
Bitter_Gourd 1
Bottle_Gourd 2
Brinjal 3
Broccoli 4
Cabbage 5
Capsicum 6
Carrot 7
Cauliflower 8
Cucumber 9
Papaya 10
Potato 11
Pumpkin 12
Radish 13
Tomato 14


In [5]:
print(classes)


{'Bean': 0, 'Bitter_Gourd': 1, 'Bottle_Gourd': 2, 'Brinjal': 3, 'Broccoli': 4, 'Cabbage': 5, 'Capsicum': 6, 'Carrot': 7, 'Cauliflower': 8, 'Cucumber': 9, 'Papaya': 10, 'Potato': 11, 'Pumpkin': 12, 'Radish': 13, 'Tomato': 14}


In [6]:
class net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(3,32,3)
        self.conv2=nn.Conv2d(32,64,3)
        self.conv3=nn.Conv2d(64,128,3)
        self.pool1 = nn.MaxPool2d((2, 2))
        self.pool2 = nn.MaxPool2d((2, 2))
        self.pool3 = nn.MaxPool2d((2, 2))
        # commenting out fc layers, replace value with our output
        self.fc1 = nn.Linear(4608, 1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, 32)
        self.fc4 = nn.Linear(32,len(classes))
        
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.conv3(x))
        x = self.pool3(x)
        x = x.flatten(start_dim=1) # flattening out
        #print(x.shape) # printing the shape of the flattened output
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.softmax(x, dim=1)
        
    


In [7]:
net = net().to(device)
#net.forward(torch.randn(1, 3, 64, 64).to(device)) # passing a sample input (random)

In [8]:
optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = nn.MSELoss()
def train(X,y, batch_size, epochs):
    for epoch in range(epochs):
        randomize = np.arange(len(y))

        np.random.shuffle(randomize)

        shuffled_X = X[randomize]
        shuffled_y = y[randomize]
        for i in range(0, len(y), batch_size):
            
            batch_X = torch.Tensor(shuffled_X[i:i+batch_size]).view(-1,3,IMG_SIZE, IMG_SIZE).float()
            batch_y = torch.Tensor(np.array(shuffled_y[i:i+batch_size]))
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            
            outputs = net(batch_X)
            batch_y = torch.nn.functional.one_hot(batch_y.to(torch.int64), num_classes=len(classes))
            loss=loss_function(outputs, batch_y.float())
           
            loss.backward()
            optimizer.step()
            net.zero_grad()
        print(f"Epoch: {epoch}. Loss: {loss}")

In [9]:

train(np_training_data, np_training_labels, 64, 10)

Epoch: 0. Loss: 0.04959617182612419
Epoch: 1. Loss: 0.050736960023641586
Epoch: 2. Loss: 0.0419231541454792
Epoch: 3. Loss: 0.027687687426805496
Epoch: 4. Loss: 0.020344510674476624
Epoch: 5. Loss: 0.021529538556933403
Epoch: 6. Loss: 0.01141120120882988
Epoch: 7. Loss: 0.021463394165039062
Epoch: 8. Loss: 0.004401558544486761
Epoch: 9. Loss: 0.009889046661555767


In [10]:
torch.save(net.state_dict(), "trainedcnn.pth")

In [11]:
test_X=[]
test_y=[]
for dirname, _, filenames in os.walk(DATA_PATH + "test/"):
    class_name = dirname.split("/")[-1]
    for filename in filenames:
        test_X.append(preprocess_image(os.path.join(dirname, filename)))
        test_y.append(classes[class_name])
        

In [12]:
test_X = torch.tensor(np.array(test_X))
test_y = torch.tensor(np.array(test_y))
test_X.to(device)
test_y.to(device)
print(test_X.shape)
def test(net):
    correct = 0
    total = 0
    with torch.no_grad():
        for i in range(len(test_X)):
            
            net_out = net(test_X[i].view(-1, 3,IMG_SIZE, IMG_SIZE).to(device))[0]  
            
            predicted_class = torch.argmax(net_out)
            
            if predicted_class == test_y[i]:
                correct += 1
            total += 1

    print("Accuracy: ", round(correct/total, 3))

test(net.double())

torch.Size([3000, 64, 64, 3])
Accuracy:  0.848
